## Apprentissage supervisé sur une Matrice Document-Terme - sujet ouvert

Je propose ici un petit template reprennant les principaux éléments du notebook "1_baseline" afin de mettre en place vos propres modèles d'analyse de sentiment sur les données.

Ne pas hésiter à faire des copies pour séparer des approches distinctes ; sinon on peut faire autant d'expériences qu'on le souhaite dans un unique notebook, mais il faut réussir à ne pas se perdre entre toutes les variables (données, modèles, scores...) que l'on instancie !

In [ ]:
# Je remets tous les imports potentiellement utiles,
# mais on peut être plus parcimonieux et/ou mieux
# organisé et mettre cette cellule à jour pour
# correspondre à tout ce qu'on utilise effectivement
# dans ce notebook.
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import sklearn

On charge les données d'entraînement et validation.<br/>
On extrait les cibles dans des variables explicitement nommées `y_[subset]`.

In [ ]:
train = pd.read_csv('data/train.tsv', sep='\t')
valid = pd.read_csv('data/valid.tsv', sep='\t')

y_train = train['polarity'] 
y_valid = valid['polarity']

On peut optionnellement faire un peu de pré-traitement sur les données.<br/>
Je remets la fonction utilisée dans le notebook baseline, qui peut être modifiée ou ignorée.

In [ ]:
def normalize_text(text):
    """Apply basic normalization to a given text."""
    tbis = re.sub(r"[^\w\s\.\-'/]", "", text.lower())  # on retire la ponctuation indésirable.
    tbis = tbis.replace(' - ', ' ')  # on ne garde que les tirets intra-mot
    tbis = tbis.replace('?', '.').replace('!', '.')  # on remplace certains signes
    tbis = re.sub('  +', ' ', tbis)  # on retire les espaces redondants
    tbis = re.sub(r'\.\.+', '\.', tbis)  # on retire les points redondants
    tbis = tbis.replace(' .', '.')  # on retire les espaces avant un point
    return tbis.strip('. ') + '.'   # on oblige à commencer par une lettre et finir par un point

In [ ]:
# Si l'on uncomment les deux lignes suivantes,
# on remplace les textes par une version nettoyée.

#train['text'] = train['text'].apply(normalize_text)
#valid['text'] = valid['text'].apply(normalize_text)

Je remets le code initial pour vectoriser le corpus en une DTM.<br/>
C'est ici qu'il faut modifier et/ou ajouter du post-traitement pour changer les _features_!

In [ ]:
# Instanciation du vectoriseur (note: on peut préférer le TfIdfVectorizer).
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(
    lowercase=True,           # on passe le texte en minuscule
    stop_words='english',     # liste existante de stopwords anglais à filtrer
    token_pattern=r"\b[\w\-']+\b",  # on ne coupe pas les unigrammes autour des tirets et apostrophes
    ngram_range=(1, 2),       # on considère les unigrammes et les bigrammes
    min_df=(100 / len(train)) # on ne conserve que les tokens apparaissant dans au moins cent documents
)

# Apprentissage et transformation sur train; transformation sur valid.
X_train = count_vect.fit_transform(train['text'])
X_valid = count_vect.transform(valid['text'])

In [ ]:
# Des idées pour sous-sélectionner les variables (tokens) retenues ?

Une fois les features établies (note: on peut en préparer des versions distinctes à tester et mettre en concurrence), on passe à la modélisation. Ici je remets l'exemple de la régression logistique, à modifier ou remplacer / étendre avec d'autres modèles.

A noter, l'[API](https://fr.wikipedia.org/wiki/Interface_de_programmation) de Scikit-Learn est la même pour toutes les classes de modèles, donc on peut réutiliser la majeur partie des instructions (`clf.fit`, `clf.predict`, etc.) quelle que soit la classe de `clf` !

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(penalty='l1', solver='liblinear')
clf.fit(X_train, y_train)

print("Evaluation sur les données d'entraînement :")
print(sklearn.metrics.classification_report(
    y_true=y_train, y_pred=clf.predict(X_train)
))
print("Evaluation sur les données de validation :")
print(sklearn.metrics.classification_report(
    y_true=y_valid, y_pred=clf.predict(X_valid)
))

Faisons un petit peu de _tuning_ sur les hyper-paramètres du modèle :

In [ ]:
from sklearn.model_selection import GridSearchCV

On commence par un grid search (avec cross-validation) sur le paramètre de régularisation pour la régression LASSO.

In [ ]:
# On définit la stratégie de grid-search.
# Ici on prend une régression logistique LASSO,
# et on teste différents paramètres "C".
grid_cv = GridSearchCV(
    estimator=LogisticRegression(penalty='l1', solver='liblinear'),
    param_grid={'C': [0.01, 0.05, 0.1, 0.25, 0.5, 1.0]},
    scoring=('accuracy', 'precision', 'recall', 'roc_auc'),
    refit=False,
    n_jobs=-1,  # on parallélise sur les CPUs disponibles
    cv=5        # 5-fold cross-validation
)

In [ ]:
# On exécute le grid search sur nos données d'entraînement
# (la validation se fait par validation croisée 5-fold).
grid_cv.fit(X_train, y_train)

In [ ]:
# On extrait les résultats sous forme d'un DataFrame.
results = pd.DataFrame(grid_cv.cv_results_)
results

In [ ]:
# Visualisons les scores selon le paramètre retenu.
cols = ['params'] + [
    '%s_test_%s' % (stat, score)
    for score in ('roc_auc', 'accuracy', 'precision', 'recall')
    for stat in ('mean', 'std')
]
results[cols].sort_values('mean_test_roc_auc', ascending=False).T

On peut tout aussi bien chercher d'un même coup si la pénalisation L2 fait l'affaire, tout en explorant les paramètres de régularisation !

In [ ]:
# On essaie deux modèles : pénalisation L1 ou L2.
# Dans chaque cas, on essaie de tuner le taux de régularisation.
param_grid = [
    {
        'penalty': ['l1'], 'solver': ['liblinear'],
        'C': [0.01, 0.05, 0.1, 0.25, 0.5, 1.0]
    },
    {
        'penalty': ['l2'], 'solver': ['lbfgs'],
        'C': [0.01, 0.05, 0.1, 0.25, 0.5, 1.0]
    }
]

# On instancie le classifieur grid-search.
grid_cv = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid=param_grid,
    scoring=('accuracy', 'precision', 'recall', 'roc_auc'),
    refit=False,
    n_jobs=-1,  # on parallélise sur les CPUs disponibles
    cv=5        # 5-fold cross-validation
)

# On fait tourner le grid-search.
grid_cv.fit(X_train, y_train)

In [ ]:
# Visualisons les résultats.
results = pd.DataFrame(grid_cv.cv_results_)
cols = ['param_penalty', 'param_C'] + [
    '%s_test_%s' % (stat, score)
    for score in ('roc_auc', 'accuracy', 'precision', 'recall')
    for stat in ('mean', 'std')
]
results[cols].sort_values('mean_test_roc_auc', ascending=False)

Finalement, on va choisir un modèle à partir de ces résultats, l'entraîner sur tout le jeu de train, et l'évaluer sur le jeu de validation.

In [ ]:
clf = LogisticRegression(penalty='l1', solver='liblinear', C=0.25)
clf.fit(X_train, y_train)

print("Evaluation sur les données d'entraînement :")
print(sklearn.metrics.classification_report(
    y_true=y_train, y_pred=clf.predict(X_train)
))
print("Evaluation sur les données de validation :")
print(sklearn.metrics.classification_report(
    y_true=y_valid, y_pred=clf.predict(X_valid)
))